In [2]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd

# Variables globales
features = []
cont_features_names = []
cate_features_names = []
cate_features_idx = []
cont_features_idx = []
encdec = None
dataset_enc = None
onehot_feature_idx = []
new_cont_idx = []

def initialize_globals(dataset, class_name):
    global features, cont_features_names, cate_features_names, cate_features_idx, cont_features_idx
    
    features = [c for c in dataset.columns if c != class_name]
    cont_features_names = list(dataset[features]._get_numeric_data().columns)
    cate_features_names = [c for c in dataset.columns if c not in cont_features_names and c != class_name]
    cate_features_idx = [features.index(f) for f in cate_features_names]
    cont_features_idx = [features.index(f) for f in cont_features_names]

def enc_fit_transform(dataset, class_name):
    global encdec, dataset_enc, onehot_feature_idx, new_cont_idx

    dataset_values = dataset[features].values
    encdec = OneHotEncoder(handle_unknown='ignore')
    dataset_enc = encdec.fit_transform(dataset_values[:, cate_features_idx]).toarray()

    onehot_feature_idx = []
    for f in cate_features_idx:
        uniques = len(np.unique(dataset_values[:, f]))
        onehot_feature_idx.extend([f + u for u in range(uniques)])
    
    new_cont_idx = list(np.arange(len(onehot_feature_idx), len(onehot_feature_idx) + len(cont_features_idx)))
    dataset_enc_complete = np.hstack((dataset_enc, dataset_values[:, cont_features_idx]))

    return dataset_enc_complete

def dec(encoded_data):
    x_cat_decoded = encdec.inverse_transform(encoded_data[:, :len(onehot_feature_idx)])
    x_cont = encoded_data[:, len(onehot_feature_idx):]

    x_res = np.hstack((x_cat_decoded, x_cont))
    return x_res

# Uso
dataset = pd.read_csv('datasets/coinfeccion.csv')
class_name = "COINFECCION"
initialize_globals(dataset, class_name)
encoded_data = enc_fit_transform(dataset, class_name)
print("Datos originales:")
print(dataset.head())
print("\nDatos codificados:")
print(encoded_data[:5, :])
decoded_data = dec(encoded_data)
print("\nDatos decodificados:")
print(decoded_data[:5])


Datos originales:
   EDAD SEXO  ESTADIA_HOSPITAL ESTADO_AL_EGRESO SALA_DE_HOSPITAL BNE  \
0    72    F                13                F              UTI  SI   
1    49    M                16                F              UTI  SI   
2    91    F                17                F              UTI  SI   
3    90    F                15                V              UTI  NO   
4    84    M                14                F              UTI  SI   

  INSUFICIENCIA_RESPIRATORIA GASTROENTERITIS DESHIDRATACION RINOFARINGITIS  \
0                         NO              NO             NO             NO   
1                         SI              NO             NO             NO   
2                         NO              NO             NO             NO   
3                         NO              SI          GRAVE             NO   
4                         NO              NO             NO             NO   

   ...  HCTO CREATININA  TGP  TGO    DD COLESTEROL    TG AC_URICO GLICEMIA  \
0 